***EN ESTE NOTEBOOK VAMOS A IMPLEMENTAR LA SEGUNDA FUNCIONALIDAD DE NUESTRO CHATBOT, AL FACILIRALE LOS EJERCICIOS AL USUARIO SEGÚN LA PARTE DEL CUERPO QUE QUIERA ENTRENAR, LE OFRECEMOS LA POSIBILIDAD DE PODER SUSTITUIR EJERCICIOS, MOSTRANDO LOS 5 EJERCICIOS MÁS SIMILARES AL EJERCICIO EN CUESTIÓN.***

In [1]:
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import json

- Definimos la función *'find_similar_exercises_by_tittle'* que se encargará de devolver los ejercicios más similares al ejercicio dado por el usuario comparando su descripción con la de los demás ejercicios, aspectos a destacar en la función : 

    - Buscar la descripción y la parte del cuerpo correspondientes al título del ejercicio

    - Limpiar la descripción del ejercicio con *'clean_description'*

    - Codificar la descripción del ejercicio con .encode()

    - Calcular similitud con otras descripciones de ejercicios
    
    - Obtener los ejercicios similares

In [2]:
# Cargar los datos de ejercicios desde un archivo JSON
with open("../../json_data/datos_ejercicios_submuestreo.json", "r", encoding="utf-8") as file:
    exercise_data = json.load(file)

# Cargar el modelo preentrenado de Sentence Transformers
model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')

def find_description_by_title(title):
    for exercise in exercise_data:
        if exercise["Title"].lower() == title.lower():
            return exercise["Desc"]
    return None

def find_similar_exercises_by_title(title, n=5):
    query_description = find_description_by_title(title)
    if query_description is None:
        print("El ejercicio no fue encontrado.")
        return []

    descriptions = [query_description] + [exercise["Desc"] for exercise in exercise_data if exercise["Title"].lower() != title.lower()]
    body_parts = [exercise["BodyPart"] for exercise in exercise_data if exercise["Title"].lower() != title.lower()]
    embeddings = model.encode(descriptions)

    query_embedding = embeddings[0].reshape(1, -1)
    description_embeddings = embeddings[1:]

    similarities = cosine_similarity(query_embedding, description_embeddings)[0]

    similar_indices = similarities.argsort()[::-1][:n]
    similar_exercises = [(exercise_data[i+1]["Title"], descriptions[i], body_parts[i], similarities[i]) for i in similar_indices]

    return similar_exercises

/home/ayala/miniconda3/envs/gym/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/ayala/miniconda3/envs/gym/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


- Proporcionamos un ejemplo de como sería el funcionamiento e imprimimos la similaridad de cada ejercicio con el ejercicio dado por el usuario, además de su título, descripción y la parte del cuerpo que trabaja

In [3]:
exercise_title = input("Introduce el nombre del ejercicio: ")
similar_exercises = find_similar_exercises_by_title(exercise_title)
print(f"Ejercicios similares al ejercicio {exercise_title} por su descripción: \n".format(exercise_title))
for title, description, body_part, similarity in similar_exercises:
    print(f"***Ejercicio***\nTítulo: {title}\nDescripción: {description}\nSimilitud: {similarity}\n")


El ejercicio no fue encontrado.
Ejercicios similares al ejercicio  por su descripción: 

